In [0]:
# Imports
import numpy as np
import tensorflow as tf
import tensorflow.keras.optimizers
import tensorflow.keras.losses

## Import and check data

In [2]:
# Mount drive to access files

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Paths
train_y_path = "/content/drive/My Drive/DD2424Files/train_y.npz"
val_y_path   = "/content/drive/My Drive/DD2424Files/val_y.npz"
test_y_path  = "/content/drive/My Drive/DD2424Files/test_y.npz"

train_X_path = "/content/drive/My Drive/DD2424Files/train_X.npz"
val_X_path   = "/content/drive/My Drive/DD2424Files/val_X.npz"
test_X_path  = "/content/drive/My Drive/DD2424Files/test_X.npz"

In [0]:
# Import training and validation data
train_y = np.load(train_y_path)['arr_0']
train_y = tf.one_hot(train_y, 8)

val_y = np.load(val_y_path)['arr_0']
val_y = tf.one_hot(val_y, 8)

train_X = np.load(train_X_path)['arr_0']
val_X = np.load(val_X_path)['arr_0']

In [0]:
# Assert assumptions about data

MEL_BINS   = 96
TIME_STEPS = 1405

def check_data(X, y):
  assert len(y.shape) == 2
  assert len(X.shape) == 3
  assert y.shape[0] == X.shape[0]
  assert X.shape[1] == MEL_BINS
  assert X.shape[2] == TIME_STEPS

check_data(val_X, val_y)
check_data(train_X, train_y)

## The model

In [0]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(96, 1405)),
  #tf.keras.layers.Dense(8, activation='relu'),
  #tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(8)
])

In [17]:
# Compiling model

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.CategoricalCrossentropy(from_logits = True)
metrics = ['accuracy']


model.compile(optimizer = optimizer, loss = loss, metrics = metrics)
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 134880)            0         
_________________________________________________________________
dense_6 (Dense)              (None, 8)                 1079048   
Total params: 1,079,048
Trainable params: 1,079,048
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
# Train model
BATCH_SIZE = 32
EPOCHS = 100
history = model.fit(train_X, train_y, batch_size=BATCH_SIZE, epochs=EPOCHS,
                        validation_data=(val_X, val_y), verbose=1)

In [0]:
# Plot results